In [1]:
import pandas as pd
import datetime
import numpy as np
import scipy.stats
from math import log10 , floor

# file 불러오기

In [2]:
# volume data 가져오기
df_vol = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/volume_data.xlsx', header=[0,1], index_col=0, engine='openpyxl')
df_vol.columns.names = ["ticker",'variable']
# 구조: ticker - name - variable
# 거래량(주)	평균발행주식수 (보통)(주)	매수수량(기관계)(주)	매도수량(기관계)(주)	매수수량(개인)(주)	매도수량(개인)(주)	매수수량(외국인계)(주)	매도수량(외국인계)(주)
df_vol.index = pd.to_datetime(df_vol.index) # 날짜 형식 변경

In [3]:
# value(거래대금) data 가져오기
# 거래대금(원)	매수대금(기관계)(만원)	매도대금(기관계)(만원)	매수대금(개인)(만원)	매도대금(개인)(만원)	매수대금(외국인계)(만원)	매도대금(외국인계)(만원)
df_val = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/volume_value_data.xlsx', header=[0,1], index_col=0, engine='openpyxl')
df_val.columns.names = ["ticker",'variable']
df_val.index = pd.to_datetime(df_val.index)     # 날짜 형식 변경

In [4]:
# 두 df 합치기
df_vol = pd.concat([df_vol, df_val], axis=1)
df_vol.sort_index(axis=1, inplace=True)         # col index 정렬 -> loc 오류 방지

## refine 전 데이터 사용 : df 

In [10]:
# result 구분
df = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AR.xlsx', engine='openpyxl') #일자, 기업
# 날짜 형식 다름
df['일자'] = df['일자'].astype(str)
df['일자'].str[-8:]
df['일자'] = pd.to_datetime(df['일자'])

In [21]:
df.sort_values(by=['일자','기업'], ascending=[False, True] ,inplace=True)

In [19]:
df = df.loc[:,['일자','제목','기업','ESG','중복 횟수']]

In [22]:
df.reset_index(inplace =True, drop=True)

In [23]:
df

,일자,제목,기업,ESG,중복 횟수
0,2021-12-31,"'택배노조 총파업' 배송 차질 타 택배사 ""CJ대한통운 물량 거부""",CJ대한통운,파업,0
1,2021-12-31,"검찰, '공정위 조사방해' 현대중공업 임직원 3명 기소 대규모 증거인멸 혐의",현대중공업,공급망,3
2,2021-12-31,현대중공업 노조 '임금 개편' 효력정지 가처분신청,현대중공업,임금,0
3,2021-12-30,택배4사 노조 “CJ대한통운 파업 이관 물량 배송 거부”,CJ대한통운,파업,1
4,2021-12-29,"민주노총 서비스연맹 전국택배노동조합 CJ대한통운 강원지부, 무기한 총파업에 돌입",CJ대한통운,파업,7
...,...,...,...,...,...
6864,2012-01-05,"명문제약 36억 리베이트 뿌려 공정위, 시정명령 과징금 1억5600만원 부과",명문제약,비리,0
6865,2012-01-05,고객정보 47만여건 해킹 삼성카드 직원 구속영장,삼성카드,개인정보,2
6866,2012-01-05,"쉰들러, 현대엘리베이터와 경영권분쟁",현대엘리베이,경영권,1
6867,2012-01-04,명문제약 불법 리베이트 `덜미` 상품권에 리스비까지,명문제약,비리,3


In [28]:
df['중복 횟수'].value_counts().loc[:5]

0    4157
1    1003
2     482
3     298
4     192
5     168
Name: 중복 횟수, dtype: int64

같은 날짜의 다른 ESG 이슈 -> 제거

### 같은 윈도우에 다른 키워드가 겹치는 경우 : df -> df2

In [24]:
# 정렬
df.sort_values(by=['기업','ESG','일자'], ascending=[True,True,False] ,inplace=True)
df.reset_index(drop=True, inplace=True) # index 초기화

In [31]:
df['save_kill'] = 'save'

i = 0
while i < df.shape[0] :
    for j in range(1,min(10,df.shape[0]-i)) :
        if (df.iloc[i]['일자'] <= df.iloc[i+j]['일자'] <= df.iloc[i]['일자'] + datetime.timedelta(days=10)) \
        & (df.iloc[i]['기업'] == df.iloc[i+j]['기업']) :
            if df.iloc[i]['중복 횟수'] > df.iloc[i+j]['중복 횟수']:
                df.loc[i, 'save_kill'] = 'save'
                df.loc[i+j, 'save_kill'] = 'kill'
            elif df.iloc[i]['중복 횟수'] < df.iloc[i+j]['중복 횟수']:
                df.loc[i, 'save_kill'] = 'kill'
                df.loc[i+j, 'save_kill'] = 'save'
            else: # 동일
                df.loc[i, 'save_kill'] = 'kill'
                df.loc[i+j, 'save_kill'] = 'kill'
    i +=1

    if i%600 == 0  :
        print(i)

df2 = df[df['save_kill']=='save']


600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600


In [34]:
df2.reset_index(drop=True, inplace=True) 
df2

,일자,제목,기업,ESG,중복 횟수,save_kill
0,2018-06-04,"BGF, SK '헬로네이처' 경영권 인수 온라인 신선식품 출사표",BGF,경영권,0,save
1,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save
2,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save
3,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save
4,2017-05-15,성세환(BNK금융지주 회장) 옥중서신 “주가조작 시도 안해”,BNK금융지주,불공정거래,0,save
...,...,...,...,...,...,...
6661,2019-06-26,신한울 원전 입찰담합 의혹 '공익제보' 경기도-효성중공업 '진실공방',효성중공업,독과점,0,save
6662,2019-06-25,"바람잘 날 없는 효성중공업, 또 입참 담합 의혹",효성중공업,독과점,1,save
6663,2021-04-16,효성첨단소재 공장서 황산 1천ℓ 누출 “인명 피해 없어”,효성첨단소재,산재,1,save
6664,2019-04-01,"효성화학노조, 9일 오후 3시부터 무기한 파업 계획",효성화학,파업,0,save


## 실적 발표 제거: df2 -> df3

In [41]:
df_anc = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/anounce.xlsx', engine='openpyxl') 
df_anc['접수일자'] = pd.to_datetime(df_anc['접수일자'])

df_kospi = pd.read_csv('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/data/KOSPI.csv',  encoding='cp949')
df_kospi['종목코드'] = df_kospi['종목코드'].astype(str).str.zfill(6) 
ticker_dict = dict(zip(df_kospi['종목명'].to_list(),df_kospi['종목코드'].to_list())) # 종목명이 key
ticker_dict2 = dict(zip(df_kospi['종목코드'].to_list(),df_kospi['종목명'].to_list())) 

df2['일자'] = pd.to_datetime(df2['일자'])
column = list(df2.columns)
df_tot = pd.DataFrame(columns=column)

stocks_name = df2['기업'].value_counts().index.to_list()
stocks = [str(ticker_dict.get(i)) for i in stocks_name]

for stock in stocks_name:
    df_anc_temp = df_anc[df_anc['종목명']==stock]
    df2_temp = df2[df2['기업']==stock]
    elim = []
    for i in range(df2_temp.shape[0]):
        elim_temp = []
        for j in range(df_anc_temp.shape[0]):
            if (df2_temp.iloc[i]['일자'] - datetime.timedelta(days=10) <= df_anc_temp.iloc[j]['접수일자'] <= df2_temp.iloc[i]['일자'] + datetime.timedelta(days=10)) :
                elim_temp.append(True)
            else:
                elim_temp.append(False)
        if True in elim_temp : # 하나라도 T가 있으면 중복되는 기사가 있으므로 T로  저장
            elim.append(True)
        else : # 전부 F
            elim.append(False)
    df2_temp['제거']=elim
    df2_temp = df2_temp[df2_temp['제거']==False]
    df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
    

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['일자'] = pd.to_datetime(df2['일자'])
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

...

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_temp['제거']=elim
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

C:\Users\Jeongseok_Bang\AppData\Local\Temp\ipykernel_12716\1227627712.py:33: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)


In [42]:
df_tot

,일자,제목,기업,ESG,중복 횟수,save_kill,제거
0,2021-04-15,"SK는 반도체, 현대차는 지배력, 한화는 경영권 승계 지배구조 개편 시동",현대차,경영권,0,save,False
1,2018-09-07,엘리엇 방식으로 하면 현대차 총수일가 경영권 흔들...일감몰아주기 규제 못피해,현대차,경영권,0,save,False
2,2018-03-29,현대차 4개 사업군으로 단순화 지배구조 개편 경영권 승계 탄력,현대차,경영권,0,save,False
3,2017-03-17,"정의선 현대차 부회장, 현대모비스 사내이사 재선임...계열사 경영권 강화",현대차,경영권,0,save,False
4,2016-04-28,현대차 노조 임시대의원대회 올 임협 요구안 윤곽 \n승진거부권 등 사측 경영권 정면...,현대차,경영권,0,save,False
...,...,...,...,...,...,...,...
5200,2019-06-27,[시그널 FOCUS] 대표에 ‘추징금 폭탄’ 삼진제약 불법 리베이트 정화 신호탄?,삼진제약,비리,0,save,False
5201,2020-01-23,"노조설립 직원 성추행 해고 삼영전자, 항소심에서 패소",삼영전자,노동권,0,save,False
5202,2015-03-25,"삼성제약공업, 오염 지하수로 식품드링크 제조 유통 적발",삼성제약,환경오염,0,save,False
5203,2019-07-29,"​남영비비안, 경영권 매각설 토종 속옷업계 위기감 확산",비비안,경영권,0,save,False


In [43]:
df3 = df_tot.copy()

# variable 정의

## Abnormal Volume
Vol.1 $v(i,t)*v(m)/v(i)*v(m,t)$ \
Vol.2 $v(i,t)/v(m,t) - v(i)/v(m)$ \
Vol.3 $v(i,t)/v(i) - v(mt)/v(m)$

In [47]:
# v_i, v_m : 영업일 기준 200일(-205~-6일) => initial 값에 의존하므로 initial의 함수로 처리

def AV_tot(vol, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산
    a = ind+initial-period
    b = ind+initial-1
    v_i = df_data.loc[a:b, [vol]].mean()
    v_m = df_kospi_vol.loc[a:b, [vol]].mean()

    seq = [] # -5일부터 +5일까지 11개의 av
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = df_data.iloc[ind+j][vol]
            v_mt = df_kospi_vol.iloc[ind_mkt+j][vol]
            av = v_it/v_i - v_mt/v_m 
#             av = (v_it*v_m)/(v_mt*v_i)
            seq.append(av[0])
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
    return seq_2

In [48]:
def AV_inv(vol, buy, sell, df_data, date):

   # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산
    a = ind+initial-period
    b = ind+initial-1
    v_i = df_data.loc[a:b,[buy,sell]].mean(axis=1).mean(axis=0)
    v_m = df_kospi_vol.loc[a:b, [vol]].mean()
    
    seq = [] # -5일부터 +5일까지 11개의 av
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음) nan으로 처리 -> 나중에 제거
            seq.append(np.nan) 
        else:
            ### null 제어하면서 계산 ###
            if np.isnan(df_data.iloc[ind+j][buy])  :    # 아무것도 없으면 0으로 처리 => 0(수식상) vs 1(개념상)
                seq.append(0)
                
            else:  
                v_it = (df_data.iloc[ind+j][buy]+df_data.iloc[ind+j][sell])/2
                v_mt = df_kospi_vol.iloc[ind_mkt+j][vol]
                av = v_it/v_i - v_mt/v_m
#                 av = (v_it*v_m)/(v_mt*v_i)
                seq.append(av[0])
            
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정

    return seq_2


## BSI

In [49]:
def BSI(buy, sell, df_data, date):

    seq = [] # -5일부터 +5일까지 11개의 bsi
    for j in range(initial, final+1):               # -5~5까지 각 날짜의 bsi 계산하는 loop

        df_temp = df_data.shift(-j)             #minus는 방향 보정

        buy_vol = df_temp.loc[date,buy]         # 해당 종목의 매수수량
        sell_vol = df_temp.loc[date,sell]       # 해당 종목의 매도수량
        
        ### null 제어하면서 BSI 계산 ###
        # 나눌 수 없는 경우: np.nan이거나 0, 0 인 경우이므로 제거
        # 상장되지 않았을 때의 사건은 제거된 상황
        if (buy_vol==0 and sell_vol==0) or (np.isnan(buy_vol) or np.isnan(sell_vol)) :    # 하나라도 nan이거나 둘다 0인 경우
            #############
            # 0 or np.nan?
            bsi = 0      
            #############
        else: 
            bsi = buy_vol/(buy_vol+sell_vol) -0.5
        seq.append(bsi)
        
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음
    
    return seq_2
    

## TurnOver

In [50]:

def TO_tot(vol, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산 (vol/out을 각각 200일 각각)
    to_mean = []
    for j in range(ind+initial-period,ind+initial):
        to_past = df_data.loc[j][vol] / df_data.loc[j]['평균발행주식수 (보통)(주)']
        to_mean.append(to_past)
    to_mean = np.array(to_mean) 
    to_i = to_mean.mean()
    
    # -5일부터 +5일까지 11개의 to
    seq = [] 
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = df_data.loc[ind+j][vol]          
            o_it = df_data.loc[ind+j]['평균발행주식수 (보통)(주)']          
            to = v_it/o_it - to_i
            seq.append(to)         
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
        
    return seq_2


In [51]:

def TO_inv(vol, buy, sell, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산 (vol/out을 각각 200일 각각)
    to_mean = []
    for j in range(ind+initial-period,ind+initial):
        if df_data.loc[j][buy] == np.nan: # or df_data.loc[j][sell] == np.nan
            to_past = 0
        else:
            to_past = (df_data.loc[j][buy] + df_data.loc[j][sell]) / (2*df_data.loc[j]['평균발행주식수 (보통)(주)'])
        to_mean.append(to_past)
    to_mean = np.array(to_mean)    
    to_i = to_mean.mean()
    
    # -5일부터 +5일까지 11개의 to
    seq = [] 
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = (df_data.loc[ind+j][buy] + df_data.loc[ind+j][sell])/2
            o_it = df_data.loc[ind+j]['평균발행주식수 (보통)(주)']          
            to = v_it/o_it - to_i
            seq.append(to)         

    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
        
    return seq_2

## var. 연산

In [53]:
# window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
window = [(-5,-2),(-1,-1),(0,0),(1,1),(2,5)]
initial = -5 
final = 5
period = 200 # 평균값을 계산하는 기간

In [56]:
# kospi volume data 가져오기
# date - ror - vol
df_kospi_vol = pd.read_csv('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/data/KOSPI_vol.csv', encoding='cp949')
df_kospi_vol['date'] = pd.to_datetime(df_kospi_vol['date']) # 날짜 형식 변경

In [77]:
# 21년12월 말은 제거
df3 = df3[df3['일자'] < datetime.datetime.strptime('2021-12-20','%Y-%m-%d')]
df3 = df3[df3['기업'] != '맥쿼리인프라']

In [78]:
df3.sort_values(by=['기업','ESG','일자'], ascending=[True,True,False] ,inplace=True)
df3.reset_index(inplace=True, drop=True)
df3

,일자,제목,기업,ESG,중복 횟수,save_kill,제거
0,2018-06-04,"BGF, SK '헬로네이처' 경영권 인수 온라인 신선식품 출사표",BGF,경영권,0,save,False
1,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False
2,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False
3,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False
4,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False
...,...,...,...,...,...,...,...
5181,2017-07-12,면세점 비리 의혹에 한화갤러리아 '울고' 호텔신라 '웃고',호텔신라,비리,0,save,False
5182,2017-06-09,"임우재 뇌물 혐의, 이부진의 호텔신라에 ‘불똥’",호텔신라,비리,1,save,False
5183,2019-09-10,"효성중공업, 국세청 382억원 추징금 부과",효성중공업,공급망,0,save,False
5184,2021-04-16,효성첨단소재 공장서 황산 1천ℓ 누출 “인명 피해 없어”,효성첨단소재,산재,1,save,False


In [79]:
df_kospi['종목코드'] = df_kospi['종목코드'].astype(str).str.zfill(6) 
name_ticker = dict(zip(df_kospi['종목명'].to_list(),df_kospi['종목코드'].to_list())) # {종목명:tikcer}
ticker_name = dict(zip(df_kospi['종목코드'].to_list(),df_kospi['종목명'].to_list())) # {ticker:종목명}


# columns 이름 설정
columns = []
ms = ['AV_VOL', 'AV_VAL','BSI','TO']
invs = ['TOT','IND', 'INS' ,'FOR'] 
col_temp = []
for m in ms:
    for inv in invs:
        col_temp.append(m+'_'+inv)
col_temp.remove('BSI_TOT')

for c in col_temp:
    for a,b in window:
        columns.append(c+'_'+str(a)+'_'+str(b))

tot = [] # 각 사건의 window별 계산결과
# 개별 사건에 대한 BSI 계산
for i in range(df3.shape[0]) : #df.shape[0]
    ticker = name_ticker.get(df3.iloc[i]['기업'])    # 해당 기업의 종목코드
    date = df3.iloc[i]['일자']                       # 해당 사건의 날짜
    
    if not (date in df_vol.index): # 해당날짜가 비영업일인 경우 다음 영업일을 계산 (vol/val 상관없음)
        while not(date in df_vol.index): # False일 때에만 작동
            date += datetime.timedelta(days=1)
            if date > datetime.datetime.strptime('2021-12-31','%Y-%m-%d'): #날짜 넘어가면 제거
                break
            
    
    df_data = df_vol.loc[:,(ticker)]        # 해당 ticker의 모든 var.에 대한 자료를 별도의 df에 저장 (single-index)        # 주의) 매도가 먼저 나옴


    window_av_tot_vol = AV_tot('거래량(주)', df_data,  date)
    window_av_ind_vol = AV_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)',df_data,  date)
    window_av_ins_vol = AV_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_av_for_vol = AV_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_av_tot_val = AV_tot('거래대금(원)', df_data,  date)
    window_av_ind_val = AV_inv('거래대금(원)','매수대금(개인)(만원)','매도대금(개인)(만원)', df_data,  date)
    window_av_ins_val = AV_inv('거래대금(원)','매수대금(기관계)(만원)',	'매도대금(기관계)(만원)', df_data,  date)
    window_av_for_val = AV_inv('거래대금(원)','매수대금(외국인계)(만원)', '매도대금(외국인계)(만원)',df_data,  date)

    window_bsi_ind = BSI('매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_bsi_ins = BSI('매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_bsi_for = BSI('매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_to_tot = TO_tot('거래량(주)', df_data,  date)
    window_to_ind = TO_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_to_ins = TO_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_to_for = TO_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_tot = window_av_tot_vol + window_av_ind_vol + window_av_ins_vol + window_av_for_vol \
        + window_av_tot_val + window_av_ind_val + window_av_ins_val + window_av_for_val \
            + window_bsi_ind + window_bsi_ins + window_bsi_for \
                + window_to_tot + window_to_ind + window_to_ins + window_to_for

    tot.append(window_tot)  # 이중리스트
    
    if i%300 == 0:
        print(round(i/df3.shape[0],3))
    
tot = np.array(tot)
df_temp = pd.DataFrame(tot, columns=columns)
df3 = pd.concat([df3, df_temp], axis=1)

0.0
0.058
0.116
0.174
0.231
0.289
0.347
0.405
0.463
0.521
0.578
0.636
0.694
0.752
0.81
0.868
0.926
0.983


In [115]:
df3

,일자,제목,기업,ESG,중복 횟수,save_kill,제거,AV_VOL_TOT_-5_-2,AV_VOL_TOT_-1_-1,AV_VOL_TOT_0_0,...,TO_INS_-5_-2,TO_INS_-1_-1,TO_INS_0_0,TO_INS_1_1,TO_INS_2_5,TO_FOR_-5_-2,TO_FOR_-1_-1,TO_FOR_0_0,TO_FOR_1_1,TO_FOR_2_5
0,2018-06-04,"BGF, SK '헬로네이처' 경영권 인수 온라인 신선식품 출사표",BGF,경영권,0,save,False,-0.923361,-0.991922,-0.307791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False,0.128097,-0.260732,-0.428205,...,-0.000249,-2.865339e-04,-0.000151,-0.000013,0.000053,-0.000307,-0.000648,-0.000780,-0.000805,-0.000126
2,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False,-0.188109,0.209962,-0.251972,...,-0.000180,-3.931712e-07,0.000044,0.000150,0.000396,-0.000444,0.000229,-0.000518,-0.000636,0.000083
3,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False,0.017436,0.168691,0.034991,...,0.000085,4.259276e-04,0.000163,0.000183,-0.000137,-0.000033,-0.000048,-0.000430,0.000862,0.000574
4,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False,0.391756,0.256898,-0.037706,...,0.000154,-1.852024e-04,-0.000287,-0.000212,0.000299,0.000415,0.000483,-0.000020,0.000152,0.000846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5181,2017-07-12,면세점 비리 의혹에 한화갤러리아 '울고' 호텔신라 '웃고',호텔신라,비리,0,save,False,-0.284601,-0.310572,1.536280,...,-0.000895,-2.711907e-04,0.001246,0.000497,0.000853,-0.001097,-0.001655,0.003976,0.001040,0.000792
5182,2017-06-09,"임우재 뇌물 혐의, 이부진의 호텔신라에 ‘불똥’",호텔신라,비리,1,save,False,-0.208108,-0.005533,-0.542903,...,-0.000786,2.695719e-04,-0.000468,-0.000535,0.000774,-0.000371,0.002079,-0.000052,-0.000761,0.001535
5183,2019-09-10,"효성중공업, 국세청 382억원 추징금 부과",효성중공업,공급망,0,save,False,-0.332025,-0.646029,-0.715219,...,-0.000786,-4.674622e-04,-0.000741,-0.000078,-0.000531,-0.000383,-0.000365,-0.000400,-0.000024,-0.000297
5184,2021-04-16,효성첨단소재 공장서 황산 1천ℓ 누출 “인명 피해 없어”,효성첨단소재,산재,1,save,False,-1.106858,-0.742890,0.799528,...,-0.000713,-5.984617e-04,0.000239,-0.000451,-0.000311,-0.003126,-0.002517,0.005686,0.002697,0.002055


In [143]:
df3.dropna(inplace=True)
df3.reset_index(inplace=True,drop=True)

In [144]:
df3

,일자,제목,기업,ESG,중복 횟수,save_kill,제거,AV_VOL_TOT_-5_-2,AV_VOL_TOT_-1_-1,AV_VOL_TOT_0_0,...,TO_INS_-5_-2,TO_INS_-1_-1,TO_INS_0_0,TO_INS_1_1,TO_INS_2_5,TO_FOR_-5_-2,TO_FOR_-1_-1,TO_FOR_0_0,TO_FOR_1_1,TO_FOR_2_5
0,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False,0.128097,-0.260732,-0.428205,...,-0.000249,-2.865339e-04,-0.000151,-0.000013,0.000053,-0.000307,-0.000648,-0.000780,-0.000805,-0.000126
1,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False,-0.188109,0.209962,-0.251972,...,-0.000180,-3.931712e-07,0.000044,0.000150,0.000396,-0.000444,0.000229,-0.000518,-0.000636,0.000083
2,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False,0.017436,0.168691,0.034991,...,0.000085,4.259276e-04,0.000163,0.000183,-0.000137,-0.000033,-0.000048,-0.000430,0.000862,0.000574
3,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False,0.391756,0.256898,-0.037706,...,0.000154,-1.852024e-04,-0.000287,-0.000212,0.000299,0.000415,0.000483,-0.000020,0.000152,0.000846
4,2017-04-19,'주가 조작 혐의' 성세환 BNK금융지주 회장 구속,BNK금융지주,불공정거래,1,save,False,-0.208828,-0.323645,0.601944,...,-0.000345,-4.478847e-04,0.000153,-0.000116,0.000117,0.000010,-0.000377,0.000841,0.000307,0.000314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,2021-01-18,'국정농단' 이재용 징역 2년6월 법정구속 호텔신라 ↑,호텔신라,비리,0,save,False,0.595515,-0.177064,3.893270,...,0.003321,1.187790e-04,0.005161,-0.000318,0.000487,0.003211,0.000088,0.002361,-0.001326,0.000366
4425,2017-07-12,면세점 비리 의혹에 한화갤러리아 '울고' 호텔신라 '웃고',호텔신라,비리,0,save,False,-0.284601,-0.310572,1.536280,...,-0.000895,-2.711907e-04,0.001246,0.000497,0.000853,-0.001097,-0.001655,0.003976,0.001040,0.000792
4426,2017-06-09,"임우재 뇌물 혐의, 이부진의 호텔신라에 ‘불똥’",호텔신라,비리,1,save,False,-0.208108,-0.005533,-0.542903,...,-0.000786,2.695719e-04,-0.000468,-0.000535,0.000774,-0.000371,0.002079,-0.000052,-0.000761,0.001535
4427,2019-09-10,"효성중공업, 국세청 382억원 추징금 부과",효성중공업,공급망,0,save,False,-0.332025,-0.646029,-0.715219,...,-0.000786,-4.674622e-04,-0.000741,-0.000078,-0.000531,-0.000383,-0.000365,-0.000400,-0.000024,-0.000297


In [145]:
df3.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_result_5.xlsx', index=None)

# t-test

In [96]:
E =['환경오염']
S= ['파업','산재','고용','임금','노동권', '독과점','공급망','상생','품질','개인정보','유해물질','광고','성범죄'] 
G = ['비리','도덕성','경영권','불공정거래','분식회계']
ESG = [E,S,G]

In [101]:
yrs = [i for i in range(2012,2022)]
yr_smp = []
for yr in yrs:
    df_yr = df3[df3['일자'].dt.year == yr]
    yr_smp.append(df_yr.shape[0])
    print(yr, df_yr.shape[0])    

print(sum(yr_smp))

for pillar in ESG:
    df_pillar = df[df['ESG'].isin(pillar)]
    print(df3[df3['ESG'].isin(pillar)].shape[0])
    for yr in yrs:
        df_yr = df_pillar[df_pillar['일자'].dt.year == yr]
        yr_smp.append(df_yr.shape[0])
        print(yr, df_yr.shape[0])  

2012 331
2013 568
2014 557
2015 527
2016 579
2017 482
2018 758
2019 571
2020 421
2021 392
5186
138
2012 3
2013 14
2014 15
2015 10
2016 15
2017 13
2018 20
2019 65
2020 22
2021 23
2978
2012 280
2013 436
2014 458
2015 381
2016 400
2017 371
2018 504
2019 413
2020 335
2021 394
2070
2012 170
2013 304
2014 250
2015 266
2016 285
2017 262
2018 490
2019 320
2020 191
2021 159


In [102]:
# window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
window = [(-5,-2),(-1,-1),(0,0),(1,1),(2,5)]
# window = [(-5,-1),(0,0),(1,5)]
initial = -5 
final = 5
period = 200 # 평균값을 계산하는 기간

In [103]:
# mean, t-test

def col_gen(ms, invs):
    col_temp = []
    for m in ms:
        for inv in invs:
            col_temp.append(m+'_'+inv)
    column = []
    for c in col_temp:
        for a,b in window:
            column.append(c+'_'+str(a)+'_'+str(b))
    return column
    

In [104]:
col_AV = col_gen(['AV_VOL', 'AV_VAL'],['TOT','IND', 'INS' ,'FOR'])
col_BSI = col_gen(['BSI'],['IND', 'INS' ,'FOR'])
col_TO = col_gen(['TO'],['TOT','IND', 'INS' ,'FOR'])

In [121]:

for col in col_AV :  
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_BSI:
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_TO:
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))

AV_VOL_TOT_-5_-2 
 mean: 0.09024 t-stats: 4.93019 p-value: 0.0
AV_VOL_TOT_-1_-1 
 mean: 0.10277 t-stats: 4.56678 p-value: 1e-05
AV_VOL_TOT_0_0 
 mean: 0.20428 t-stats: 6.77497 p-value: 0.0
AV_VOL_TOT_1_1 
 mean: 0.15605 t-stats: 5.41794 p-value: 0.0
AV_VOL_TOT_2_5 
 mean: 0.06814 t-stats: 3.33612 p-value: 0.00086
AV_VOL_IND_-5_-2 
 mean: 0.1516 t-stats: 5.16389 p-value: 0.0
AV_VOL_IND_-1_-1 
 mean: 0.17879 t-stats: 6.25632 p-value: 0.0
AV_VOL_IND_0_0 
 mean: 0.3172 t-stats: 7.46395 p-value: 0.0
AV_VOL_IND_1_1 
 mean: 0.26099 t-stats: 6.07247 p-value: 0.0
AV_VOL_IND_2_5 
 mean: 0.14675 t-stats: 5.09772 p-value: 0.0
AV_VOL_INS_-5_-2 
 mean: 0.04556 t-stats: 3.15019 p-value: 0.00164
AV_VOL_INS_-1_-1 
 mean: 0.04351 t-stats: 2.37795 p-value: 0.01745
AV_VOL_INS_0_0 
 mean: 0.12065 t-stats: 4.85165 p-value: 0.0
AV_VOL_INS_1_1 
 mean: 0.1352 t-stats: 3.514 p-value: 0.00045
AV_VOL_INS_2_5 
 mean: 0.0503 t-stats: 1.87892 p-value: 0.06032
AV_VOL_FOR_-5_-2 
 mean: 0.06053 t-stats: 4.90259 p-value

### 엑셀로 정리

In [128]:
def col_gen_m(ms, invs):
    col_temp = []
    for m in ms:
        for inv in invs:
            col_temp.append(m+'_'+inv)
    return col_temp

In [129]:
col_m = col_gen_m(['AV_VOL', 'AV_VAL','BSI','TO'],['TOT','IND', 'INS' ,'FOR'])
col_m.remove('BSI_TOT')

In [130]:
# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        m= df3[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
        
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [131]:
df_result

AV_VOL_TOT        AV_VOL_IND        AV_VOL_INS        AV_VOL_FOR  \
               mean t_stat       mean t_stat       mean t_stat       mean   
(-5, -2)  0.0902***   4.93   0.152***   5.16  0.0456***   3.15  0.0605***   
(-1, -1)   0.103***   4.57   0.179***   6.26   0.0435**   2.38  0.0599***   
(0, 0)     0.204***   6.77   0.317***   7.46   0.121***   4.85   0.149***   
(1, 1)     0.156***   5.42   0.261***   6.07   0.135***   3.51   0.083***   
(2, 5)    0.0681***   3.34   0.147***   5.10    0.0503*   1.88     0.0143   

                AV_VAL_TOT         ...      BSI_FOR              TO_TOT  \
         t_stat       mean t_stat  ...         mean t_stat         mean   
(-5, -2)   4.90   0.104***   5.64  ...     -0.00214  -1.38  0.000596***   
(-1, -1)   2.97   0.117***   4.94  ...     -0.00103  -0.44  0.000943***   
(0, 0)     4.60   0.223***   6.95  ...      0.00196   0.84   0.00166***   
(1, 1)     4.75   0.191***   5.31  ...      0.00126   0.54    0.0009***   
(2, 5)     1.30  0.0982***   3.70  ...  -0.00526***  -3.33   0.000332**   

                      TO_IND              TO_INS              TO_FOR         
         t_stat         mean t_stat         mean t_stat         mean t_stat  
(-5, -2)   3.68   0.00048***   3.31  3.68e-05***   2.92  8.16e-05***   5.34  
(-1, -1)   3.31  0.000794***   3.01  6.09e-05***   3.06  9.72e-05***   3.95  
(0, 0)     3.75   0.00134***   3.24  0.000113***   5.27  0.000207***   4.57  
(1, 1)     4.01  0.000718***   3.51  7.61e-05***   4.35  0.000113***   5.53  
(2, 5)     2.42   0.000292**   2.37     1.16e-05   1.07   2.72e-05**   2.21  

[5 rows x 30 columns]

## winsorize

In [136]:
############# winsorize ##############
def wins(df, col):
    df_wins = df.copy()
    df_wins.sort_values(by=[col], inplace=True)
    df_wins.reset_index(inplace=True, drop=True)
    cut = int(df_wins.shape[0]*0.05)
    for i in range(cut):
        df_wins.loc[i,col] = df_wins.loc[cut,col]
        df_wins.loc[df_wins.shape[0]-1-i,col] = df_wins.loc[df_wins.shape[0]-1-cut,col]
    return df_wins

In [146]:
############# winsorize ##############
# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        ### winsor ####
        m= wins(df3,col)[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df3,col)[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df3,col)[col], 0)
        ###############
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [147]:
df_result

AV_VOL_TOT        AV_VOL_IND         AV_VOL_INS         AV_VOL_FOR  \
                mean t_stat       mean t_stat        mean t_stat        mean   
(-5, -2)    -0.00073  -0.10   0.0192**   2.04  -0.0203***  -2.68      0.0117   
(-1, -1)    -0.0171*  -1.92    0.00776   0.71  -0.0493***  -5.31    -0.00839   
(0, 0)     0.0457***   4.64  0.0792***   6.55    0.000626   0.06   0.0407***   
(1, 1)        0.0129   1.35  0.0355***   3.09    -0.0175*  -1.82        0.01   
(2, 5)    -0.0271***  -3.49   -0.00193  -0.20  -0.0384***  -5.05  -0.0218***   

                 AV_VAL_TOT         ...     BSI_FOR               TO_TOT  \
         t_stat        mean t_stat  ...        mean t_stat          mean   
(-5, -2)   1.53     0.00285   0.41  ...    -0.00217  -1.57     -3.76e-05   
(-1, -1)  -0.88    -0.019**  -2.36  ...    -0.00104  -0.50  -0.000133***   
(0, 0)     4.05   0.0338***   3.77  ...     0.00214   1.01   0.000148***   
(1, 1)     1.05     0.00511   0.60  ...    0.000969   0.46     -2.62e-05   
(2, 5)    -2.80  -0.0263***  -3.78  ...  -0.0054***  -3.87  -0.000149***   

                       TO_IND               TO_INS              TO_FOR         
         t_stat          mean t_stat          mean t_stat         mean t_stat  
(-5, -2)  -1.10   -5.41e-05**  -2.36    -1.18e-05*  -1.87   1.79e-05**   2.29  
(-1, -1)  -3.30  -9.84e-05***  -3.84   -3.1e-05***  -3.86    -2.24e-06  -0.22  
(0, 0)     3.32   8.85e-05***   3.06      1.24e-05   1.42  4.15e-05***   3.93  
(1, 1)    -0.63    -5.8e-05**  -2.20     -5.42e-06  -0.66    1.66e-05*   1.68  
(2, 5)    -4.34  -0.000129***  -5.77  -2.28e-05***  -3.55   -1.43e-05*  -1.89  

[5 rows x 30 columns]

In [140]:
df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_5_wins.xlsx')

## 중복 횟수 0회 제거: df3 -> df4

In [189]:
ov = 2
df4 = df3[df3['중복 횟수']>ov]

In [190]:
df4.reset_index(inplace =True, drop=True)

In [191]:
print(df4)
print(df4.shape)

            일자                                       제목       기업    ESG 중복 횟수  \
0   2017-03-07                   ‘시세조종’ 혐의 BNK금융지주 압수수색  BNK금융지주  불공정거래     5   
1   2018-03-08           부산은행 채용비리 혐의 BNK금융지주 박재경 사장 구속  BNK금융지주     비리     5   
2   2018-03-01        '채용비리 의혹' BNK금융지주 사장 등 2명 구속영장 청구  BNK금융지주     비리     3   
3   2020-12-06       CJ대한통운 롯데택배 등 농산물 운송용역 담합 혐의로 檢 고발   CJ대한통운    독과점     3   
4   2020-07-13  18년간 철강제품 운송 입찰 담합 CJ대한통운 등 7곳 과징금 460억   CJ대한통운    독과점    11   
..         ...                                      ...      ...    ...   ...   
816 2017-04-07            현대차, '엔진 결함' 세타2 차량 17만여 대 리콜      현대차     품질    19   
817 2016-10-23               현대차 '선루프 결함' 쏘나타 국내서도 리콜한다      현대차     품질     4   
818 2016-10-10                 검찰, 에어백 결함 미신고 현대차 수사 착수      현대차     품질     5   
819 2015-09-26        리콜 비상! 현대차, 미국서 쏘나타 엔진 결함 47만대 리콜      현대차     품질     3   
820 2013-12-10                 미국 '에어백 결함' 현대차 추가 리콜 검토      현대차     품질     3   

    save_kill     제거  AV_VO

In [192]:
############# winsorize
for col in col_AV :  
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_BSI:
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_TO:
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))

AV_VOL_TOT_-5_-2 
 mean: -0.00146 t-stats: -0.0962 p-value: 0.92339
AV_VOL_TOT_-1_-1 
 mean: -0.04488 t-stats: -2.28398 p-value: 0.02263
AV_VOL_TOT_0_0 
 mean: 0.06856 t-stats: 3.00423 p-value: 0.00274
AV_VOL_TOT_1_1 
 mean: -0.0122 t-stats: -0.61691 p-value: 0.53747
AV_VOL_TOT_2_5 
 mean: -0.03964 t-stats: -2.401 p-value: 0.01657
AV_VOL_IND_-5_-2 
 mean: 0.01579 t-stats: 0.84041 p-value: 0.40092
AV_VOL_IND_-1_-1 
 mean: -0.00777 t-stats: -0.30963 p-value: 0.75692
AV_VOL_IND_0_0 
 mean: 0.11873 t-stats: 4.1583 p-value: 4e-05
AV_VOL_IND_1_1 
 mean: 0.04039 t-stats: 1.58702 p-value: 0.11289
AV_VOL_IND_2_5 
 mean: 0.00157 t-stats: 0.07166 p-value: 0.94289
AV_VOL_INS_-5_-2 
 mean: -0.04379 t-stats: -2.77655 p-value: 0.00562
AV_VOL_INS_-1_-1 
 mean: -0.10148 t-stats: -5.03988 p-value: 0.0
AV_VOL_INS_0_0 
 mean: 0.00737 t-stats: 0.32127 p-value: 0.74808
AV_VOL_INS_1_1 
 mean: -0.06877 t-stats: -3.36578 p-value: 0.0008
AV_VOL_INS_2_5 
 mean: -0.06673 t-stats: -4.14386 p-value: 4e-05
AV_VOL_FO

In [193]:
############# overlap ##############
############# winsorize ##############

# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        ### winsor ####
        m= wins(df4,col)[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
        ###############
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [194]:
df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_5_ov{}_wins.xlsx'.format(ov))